In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, lower, regexp_replace, split, to_date
spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()

In [21]:
authors = spark.read.csv("file:///N/project/mag/mag-2021-01-05/mag/Authors.txt",\
                         header = False,\
                         sep=r'\t')
authors.printSchema()
authors.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)

+----+-----+--------------------+--------------------+---------+---+---+---+----------+
| _c0|  _c1|                 _c2|                 _c3|      _c4|_c5|_c6|_c7|       _c8|
+----+-----+--------------------+--------------------+---------+---+---+---+----------+
| 584|19419|gozde ozdikmenlid...|Gözde Özdikmenli-...| 79946792|  5|  5| 11|2016-06-24|
| 859|19820|           gy tolmar|          Gy. Tolmár|     null|  3|  3|  2|2016-06-24|
| 978|17891|      ximena faundez|      Ximena Faúndez|149744451| 19| 19| 62|2016-06-24|
|1139|19630|      jennifer putzi|      Jennifer Putzi|     null|  4|  4|  6|2016-06-24|
|1476|21131|           勲矢 手島|           勲矢 手島|     null|  1|  1

In [22]:
def clean_text(c):
  c = regexp_replace(c, '"' , '')
  c = regexp_replace(c, '\\\\' , '')
  #c = regexp_replace(c, "(https?\://)\S+", "")
  #c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  #c = split(c, "\\s+") tokenization...
  return c

In [23]:
authors = authors.select(
    authors._c0.cast("string").alias("authorId"),
    authors._c1.cast("int").alias("rank"),
    authors._c2.cast("string").alias("normalizedName"),
    clean_text(col("_c3")).alias("displayName"),
    authors._c4.cast("string").alias("lastKnownAffiliationId"),
    authors._c5.cast("int").alias("paperCount"),
    authors._c7.cast("int").alias("citationCount"), 
    to_date(authors._c8, "yyyy-MM-dd").alias("createdDate")
    )

In [24]:
authors = authors.select(
        authors.authorId,
        authors.rank,
        authors.normalizedName, 
        authors.displayName, 
        authors.lastKnownAffiliationId, 
        authors.paperCount, 
        authors.citationCount, 
        clean_text(col("createdDate")).alias("createdDate")
    )

In [5]:
#authors.printSchema()

In [6]:
#authors.show(10)

In [25]:
%%time

authors.coalesce(64).write.option("header","true") \
                          .option("sep","\t") \
                          .option("quoteAll", False) \
                          .option("emptyValue", None) \
                          .option("nullValue", None)\
                          .mode("overwrite") \
                          .csv('/N/project/mag/mag_jg_2021_update/nodes/authors') 

CPU times: user 36 ms, sys: 24.2 ms, total: 60.2 ms
Wall time: 2min 56s
